In [1]:
# Import necessary libraries and classes

import numpy as np
import sys
import os

from Code.DNN.layers.sigmoid import SigmoidActivation
from Code.DNN.functions.metrics import mse, accuracy
from Code.DNN.networks.neuralnet import NeuralNetwork
from Code.DNN.functions.mse import MeanSquaredError
from Code.DNN.layers.dense import DenseLayer
from Code.DNN.layers.dropout import DropOutLayer
from Code.DNN.optimizations.retained_gradient import RetGradient
from Code.DNN.optimizations.l1_reg import L1Reg
from Code.DNN.optimizations.l2_reg import L2Reg
from Code.DNN.functions.bce import BinaryCrossEntropy

from Code.utils.dataset import Dataset
import Code.utils.store_model as store_model

In [2]:
# Set random seed to control randomness

np.random.seed(42)

In [3]:
# Load datasets

dataset = Dataset('../../Dataset/DatasetsGerados/dataset_training_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_training_output.csv',
                  '../../Dataset/DatasetsGerados/dataset_validation_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_validation_output.csv',
                  '../../Dataset/DatasetsGerados/dataset_test_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_test_output.csv')

# Remover pontuação deu pior resultado
X_train, y_train, X_validation, y_validation, X_test, y_test, ids = dataset.get_datasets('Text', 'Label', sep='\t', rem_punctuation=False)

In [4]:
# Create network topology

optimizer = RetGradient(learning_rate=0.01, momentum=0.90)
loss = BinaryCrossEntropy()

regulator = L2Reg(l2_val=0.001)
net = NeuralNetwork(epochs=15, batch_size=16, optimizer=optimizer, regulator=regulator, verbose=True, loss=loss,
                    metric=accuracy, patience=2, min_delta=0.001)

n_features = X_train.shape[1]
net.add(DenseLayer(6, (n_features,)))
net.add(SigmoidActivation())
net.add(DropOutLayer(3, 0.5, (n_features,)))
net.add(DenseLayer(1))
net.add(SigmoidActivation())

In [5]:
# Train network

net.fit(X_train, y_train, X_val=X_validation, y_val=y_validation)

# Plot learning curves

net.plot_train_curves()

Epoch 1/15 - loss: 2530.0456 - accuracy: 0.7937
Epoch 2/15 - loss: 903.6789 - accuracy: 0.9512
Epoch 3/15 - loss: 767.8343 - accuracy: 0.9603
Epoch 4/15 - loss: 426.5263 - accuracy: 0.9802
Epoch 5/15 - loss: 461.2598 - accuracy: 0.9812
Epoch 6/15 - loss: 317.3355 - accuracy: 0.9883
Epoch 7/15 - loss: 292.9339 - accuracy: 0.9863
Epoch 8/15 - loss: 323.1698 - accuracy: 0.9862
Epoch 9/15 - loss: 266.2313 - accuracy: 0.9900
Epoch 10/15 - loss: 260.1271 - accuracy: 0.9893
Epoch 11/15 - loss: 213.5485 - accuracy: 0.9900
Epoch 12/15 - loss: 286.6207 - accuracy: 0.9887
Epoch 13/15 - loss: 178.4953 - accuracy: 0.9920
Epoch 14/15 - loss: 148.5933 - accuracy: 0.9947
Epoch 15/15 - loss: 165.9035 - accuracy: 0.9938


In [6]:
# Predict test dataset

out = net.predict(X_test)

if y_test is not None:
    print(net.score(y_test, out))

0.9955


In [7]:
# Store results

results_filepath = './Results/dnn_results.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(results_filepath), exist_ok=True)

results = dataset.merge_results(ids, out)
results.to_csv(results_filepath, sep='\t', index=False)

In [8]:
# Store network model

model_filepath = './Model/dnn'
model_key = 'dnn'

store_model.store_model(model_filepath, model_key, net)

In [9]:
# Retrieve network model

model_filepath = './Model/dnn'
model_key = 'dnn'

net = store_model.retrieve_model(model_filepath, model_key)